# Overview

This a notebook that inspects the results of a WarpX simulation.

# Instructions

Execute the cells below one by one, by selecting them with your mouse and typing `Shift + Enter`

In [ ]:
# General imports
import glob, sys
# Import numpy and matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib
# Import yt
import yt
yt.funcs.mylog.setLevel(50) # Deactivate yt verbose

# Import interactive notebook feature
#from IPython.display import clear_output
from ipywidgets import interact, IntSlider, RadioButtons

# Find output iterations
file_list = glob.glob('plt?????')
iterations = [ int(file_name[3:]) for file_name in file_list ]

# Functions to extract and plot the fields

In [ ]:
def extra_data( field, iteration ):
    """
    Extract the 3D array that corresponds to `field` at the 
    requested iteration, and returns as a `YTarray`, which 
    can be used exactly like a numpy array.
    """
    ds = yt.load( './plt%05d/' %iteration )
    data = ds.covering_grid(level=0, 
            left_edge=ds.domain_left_edge, 
            dims=ds.domain_dimensions)
    return( data[field] ) 

In [ ]:
def plot_field( iteration, field, slicing_direction='y' ):
    # Get the data
    field_array = extra_data( field, iteration )
    
    # Get the position of the edges
    ds = yt.load( './plt%05d/' %iteration )
    left_edge = ds.domain_left_edge.convert_to_mks()*1.e6
    right_edge = ds.domain_right_edge.convert_to_mks()*1.e6

    if slicing_direction == 'x':
        n = int( ds.domain_dimensions[0]//2 )
        data2d = field_array[n, :, :]
        extent = [ left_edge[2], right_edge[2], left_edge[1], right_edge[1] ]
    elif slicing_direction == 'y':
        n = int( ds.domain_dimensions[1]//2 )
        data2d = field_array[:, n, :]
        extent = [ left_edge[2], right_edge[2], left_edge[0], right_edge[0] ]
    elif slicing_direction == 'z':
        n = int( ds.domain_dimensions[2]//2 )
        data2d = field_array[:, :, n]
        extent = [ left_edge[1], right_edge[1], left_edge[0], right_edge[0] ]
    plt.clf()
    plt.imshow( data2d, interpolation='nearest', cmap='viridis',
              origin='lower', extent=extent )
    plt.colorbar()

# Interactive viewer

In [ ]:
interact(plot_field, 
         iteration = IntSlider(min=min(iterations), max=max(iterations), step=iterations[1]-iterations[0]),
         field = RadioButtons( options=['jx', 'jy', 'jz', 'Ex', 'Ey', 'Ez'], value='jz'),
         slicing_direction = RadioButtons( options=[ 'x', 'y', 'z'], value='y') )